<a href="https://colab.research.google.com/github/amirsalhuv/MassDetector/blob/main/Final_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.2 Imports

In [ ]:
# For image manipulation
import cv2
from PIL import Image
import numpy as np
import pandas as pd

# Image loading saving, copy ,measuring time
import time, json, os
import datetime
import pickle
# For plots
import matplotlib.pyplot as plt
import shutil
from tqdm import tqdm
import random

# Pytorch
import torch
from torch.utils.data import DataLoader,Dataset
from torch.nn import BCELoss
from torchvision.transforms import Compose, ToTensor, Normalize
from torch import nn,optim
import torch.nn.functional as F
from torchvision import models
from torchvision import transforms, datasets
from sklearn.model_selection import train_test_split
from torchvision.models import vgg16_bn, VGG16_BN_Weights

# 1.3 Definitions


In [ ]:
# Amir Salhuv:
dataset_dir = '/content/drive/MyDrive/Project_OpenU/CBIS-DDSM/'

# Yonatan Salhuv:
#dataset_dir = '/content/drive/MyDrive/'

training_folder = dataset_dir + "Full_model/"
models_folder = training_folder+'Models/'
data_folder = dataset_dir + 'Latest_model_data/'
run_mode = "evaluate_model"

# Classes and batch size
n_classes=2
batch_size = 16

# 1 - models definitions

In [ ]:
  # Clone YOLOv5 repository
  !git clone -q https://github.com/ultralytics/yolov5.git

  # for running the train.py from the yolov5 directory
  %cd yolov5

  # Install dependencies
  !pip install -q -r requirements.txt

/content/yolov5/yolov5/yolov5/yolov5/yolov5/yolov5/yolov5


In [ ]:
  def double_conv(in_channels, out_channels):
      return nn.Sequential(
          nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(out_channels),
          nn.ReLU(inplace=True),
          nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(out_channels),
          nn.ReLU(inplace=True)
      )


  def up_conv(in_channels, out_channels):
      return nn.ConvTranspose2d(
          in_channels, out_channels, kernel_size=2, stride=2
      )


  class VGGUnet(nn.Module):
    #  VGG-16 (with BN) encoder.

      def __init__(self, encoder, *, pretrained=False, out_channels=1):
          super().__init__()

          self.encoder = encoder(pretrained=pretrained).features
          self.block1 = nn.Sequential(*self.encoder[:6])
          self.block2 = nn.Sequential(*self.encoder[6:13])
          self.block3 = nn.Sequential(*self.encoder[13:20])
          self.block4 = nn.Sequential(*self.encoder[20:27])
          self.block5 = nn.Sequential(*self.encoder[27:34])

          self.bottleneck = nn.Sequential(*self.encoder[34:])
          self.conv_bottleneck = double_conv(512, 1024)

          self.up_conv6 = up_conv(1024, 512)
          self.conv6 = double_conv(512 + 512, 512)
          self.up_conv7 = up_conv(512, 256)
          self.conv7 = double_conv(256 + 512, 256)
          self.up_conv8 = up_conv(256, 128)
          self.conv8 = double_conv(128 + 256, 128)
          self.up_conv9 = up_conv(128, 64)
          self.conv9 = double_conv(64 + 128, 64)
          self.up_conv10 = up_conv(64, 32)
          self.conv10 = double_conv(32 + 64, 32)
          self.conv11 = nn.Conv2d(32, out_channels, kernel_size=1)

      def forward(self, x):
          block1 = self.block1(x)
          block2 = self.block2(block1)
          block3 = self.block3(block2)
          block4 = self.block4(block3)
          block5 = self.block5(block4)

          bottleneck = self.bottleneck(block5)
          x = self.conv_bottleneck(bottleneck)

          x = self.up_conv6(x)
          x = torch.cat([x, block5], dim=1)
          x = self.conv6(x)

          x = self.up_conv7(x)
          x = torch.cat([x, block4], dim=1)
          x = self.conv7(x)

          x = self.up_conv8(x)
          x = torch.cat([x, block3], dim=1)
          x = self.conv8(x)

          x = self.up_conv9(x)
          x = torch.cat([x, block2], dim=1)
          x = self.conv9(x)

          x = self.up_conv10(x)
          x = torch.cat([x, block1], dim=1)
          x = self.conv10(x)

          x = self.conv11(x)

          x = torch.sigmoid(x)

          return x

# 2- Import the models and input data



In [ ]:
##### Load Yolo model ######
weights_path = models_folder + 'yolo.pt'
yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path,force_reload=True)

##### Load Patch model  ######
patch_model = VGGUnet(models.vgg16_bn, pretrained=True, out_channels=1)

# Load the model weights
patch_model.load_state_dict(torch.load(models_folder + 'Patch_model_padding_2_classes.pth',map_location=torch.device('cpu')))

##### input data #####
X_train = np.load(data_folder + 'X_train.npy')
y_train = np.load(data_folder + 'y_train.npy')
X_val = np.load(data_folder + 'X_val.npy')
y_val = np.load(data_folder + 'y_val.npy')

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-7-20 Python-3.10.6 torch-2.0.1+cu118 CPU

Fusing layers... 
YOLOv5x summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
Adding AutoShape... 


## 5.1 Define consolidated model

In [ ]:
class CombinedModel(nn.Module):
    def __init__(self, yolo_model, patch_model):
        super().__init__()
        self.yolo_model = yolo_model.eval()
        self.patch_model = patch_model.eval()
        self.test_image_transforms = transforms.Compose([
              transforms.Resize((224, 224)),
              transforms.ToTensor(),
              transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
          ])

    def forward(self, input):
        predicted_mask  = np.zeros((input.shape[0],input.shape[1]),dtype=np.uint8)

        with torch.no_grad():  # Disables gradient calculation to save memory
            yolo_out = self.yolo_model(input)

            # Get all bouding boxes
            for *box, _, _ in yolo_out.pred[0]:
              # Get the bounding box (and normilize them)
              x, y, w, h = (int(box[0]),int(box[1]),int(box[2]),int(box[3]))

              # Cut the input according to the window
              patch = input[y:h,x:w,:]

              # Resize according to the input for the 2nd model
              resized_patch = cv2.resize(patch,(256,256))

              # Convert the numpy array to a PIL Image
              output = Image.fromarray(resized_patch.astype('uint8'))

              # Convert to patch input shape
              output = self.test_image_transforms(output)
              output = torch.unsqueeze(output,0)

              # Inference from patch model
              output = self.patch_model(output)


              # Revert back to the mask size
              output = torch.squeeze(output)
              output = (output.cpu().numpy()>0.5)
              output = output.astype(np.uint8)

              # return back to original size
              output = cv2.resize(output,(w-x,h-y))

              # Add the output to the total detected mask
              predicted_mask[y:h,x:w]= output


            '''
            fig, ax = plt.subplots(2, 3, figsize=(6, 6))
            ax = ax.flatten()
            ax[0].imshow(patch)
            ax[0].title.set_text('patch')
            ax[1].imshow(input,cmap = 'gray')
            ax[1].title.set_text('input')
            ax[2].imshow(resized_patch,cmap = 'gray')
            ax[2].title.set_text('resized_patch')
            ax[3].imshow(targets,cmap = 'gray')
            ax[3].title.set_text('GT')
            ax[4].imshow(predicted_mask,cmap = 'gray')
            ax[4].title.set_text('predicted_mask')
            plt.tight_layout()
            plt.show()
            '''

        return predicted_mask


## 5.3 Metrics

In [ ]:
def dice_score(output, target, smooth=1e-6):
  output = output.cpu().numpy() > 0.5
  target = target.cpu().numpy() > 0.5
  intersection = (output & target).sum((1,2,3))
  union = (output | target).sum((1,2,3))
  dice = (2. * intersection + smooth) / (output.sum((1,2,3)) + target.sum((1,2,3)) + smooth)
  return dice

def iou_score(output, target):
  smooth = 1e-6
  #if torch.is_tensor(output):
  #    output = torch.sigmoid(output).data.cpu().numpy()
  #if torch.is_tensor(target):
  #    target = target.data.cpu().numpy()
  output = (output > 0.5)*1
  target = 1*(target > 0.5)
  intersection = (output & target).sum()
  union = (output | target).sum()
  return (intersection + smooth) / (union + smooth),intersection

## 5.5 Evaluate model

### 5.5.2 evaluate the model

In [ ]:
if run_mode == 'evaluate_model':
  %matplotlib inline
  # Images location
  val_image_dir = data_folder +'images/val'
  val_mask_dir = data_folder + 'masks/val'
  val_list = sorted(os.listdir(val_image_dir))


  # Specify device
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  # Assume that we are on a CUDA machine, then this should print a CUDA device:
  print(f'Working on device={device}')

  # Create the model and transfer to device
  model = CombinedModel(yolo_model,patch_model)
  model.to(device)

  iou_scores = []

  with torch.no_grad():
      for i,image in tqdm(enumerate(val_list[:])):
          # Read and prepare the mask for the model output
          target = cv2.imread(os.path.join(val_mask_dir, image), cv2.IMREAD_GRAYSCALE)* 255

          # Read the image and send to the model
          img = cv2.imread(os.path.join(val_image_dir, image))
          output = model(img)

          iou,intersection = iou_score(output, target)
          iou_scores.append(iou)

Working on device=cpu


209it [09:29,  2.73s/it]


In [ ]:
iou_scores = np.array(iou_scores)
iou2 = iou_scores[iou_scores>0.1]
print(f'Average IoU score on validation set: {(np.mean(iou_scores))}, iou2: {np.mean(iou2)}')


In [ ]:
print(np.sum(np.array(iou_scores)<0.1)/len(val_list)*100)

In [ ]:
print(iou_scores)